In [1]:
import mysql.connector as db
import pandas as pd
from faker import Faker
import random
from datetime import timedelta

In [3]:
# Create connection
cnx = db.connect(
    host="localhost",
    port=3306,
    user="root",
    password="root"
)

db_curr = cnx.cursor()

# Create database if it doesn't exist
db_curr.execute("CREATE DATABASE IF NOT EXISTS zomato")

# Connect to the database
cnx = db.connect(
    host="localhost",
    port=3306,
    user="root",
    password="root",
    database="zomato",
    autocommit=True
)

db_curr = cnx.cursor()

Table creations

In [4]:
db_curr.execute("""CREATE TABLE customers (
    customer_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50) NOT NULL,
    email VARCHAR(50) NOT NULL UNIQUE,
    phone VARCHAR(15) NOT NULL UNIQUE,
    location VARCHAR(100) NOT NULL,
    signup_date DATE NOT NULL,
    is_premium BOOLEAN NOT NULL,
    preferred_cuisine VARCHAR(15) NOT NULL,
    total_orders INT NOT NULL DEFAULT 0,
    average_rating INT NOT NULL
);""")
db_curr.execute("""DESC customers;""")
print(db_curr.fetchall())

[('customer_id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('name', 'varchar(50)', 'NO', '', None, ''), ('email', 'varchar(50)', 'NO', 'UNI', None, ''), ('phone', 'varchar(15)', 'NO', 'UNI', None, ''), ('location', 'varchar(100)', 'NO', '', None, ''), ('signup_date', 'date', 'NO', '', None, ''), ('is_premium', 'tinyint(1)', 'NO', '', None, ''), ('preferred_cuisine', 'varchar(15)', 'NO', '', None, ''), ('total_orders', 'int', 'NO', '', '0', ''), ('average_rating', 'int', 'NO', '', None, '')]


In [5]:
db_curr.execute("""CREATE TABLE restaurants (
    restaurant_id VARCHAR(20) PRIMARY KEY,
    name VARCHAR(50) NOT NULL,
    contact_number VARCHAR(15) NOT NULL UNIQUE,
    location VARCHAR(100) NOT NULL,
    owner_name VARCHAR(50) NOT NULL,
    average_delivery_time INT NOT NULL CHECK (average_delivery_time > 0),
    is_active BOOLEAN NOT NULL,
    cuisine_type VARCHAR(15) NOT NULL,
    total_orders INT NOT NULL DEFAULT 0,
    rating INT NOT NULL
);""")
db_curr.execute("""DESC restaurants;""")
print(db_curr.fetchall())

[('restaurant_id', 'varchar(20)', 'NO', 'PRI', None, ''), ('name', 'varchar(50)', 'NO', '', None, ''), ('contact_number', 'varchar(15)', 'NO', 'UNI', None, ''), ('location', 'varchar(100)', 'NO', '', None, ''), ('owner_name', 'varchar(50)', 'NO', '', None, ''), ('average_delivery_time', 'int', 'NO', '', None, ''), ('is_active', 'tinyint(1)', 'NO', '', None, ''), ('cuisine_type', 'varchar(15)', 'NO', '', None, ''), ('total_orders', 'int', 'NO', '', '0', ''), ('rating', 'int', 'NO', '', None, '')]


In [8]:
db_curr.execute("""CREATE TABLE orders (
    order_id INT PRIMARY KEY AUTO_INCREMENT,
    customer_id INT NOT NULL,
    restaurant_id VARCHAR(20),
    order_date DATETIME NOT NULL,
    delivery_time DATETIME NOT NULL,
    status VARCHAR(15) NOT NULL,
    total_amount FLOAT NOT NULL,
    payment_mode VARCHAR(15),
    discount_applied FLOAT NOT NULL DEFAULT 0,
    feedback_rating INT NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id) ON DELETE CASCADE,
    FOREIGN KEY (restaurant_id) REFERENCES restaurants(restaurant_id) ON DELETE SET NULL
);""")
db_curr.execute("""DESC orders;""")
print(db_curr.fetchall())

[('order_id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('customer_id', 'int', 'NO', 'MUL', None, ''), ('restaurant_id', 'varchar(20)', 'YES', 'MUL', None, ''), ('order_date', 'datetime', 'NO', '', None, ''), ('delivery_time', 'datetime', 'NO', '', None, ''), ('status', 'varchar(15)', 'NO', '', None, ''), ('total_amount', 'float', 'NO', '', None, ''), ('payment_mode', 'varchar(15)', 'YES', '', None, ''), ('discount_applied', 'float', 'NO', '', '0', ''), ('feedback_rating', 'int', 'NO', '', None, '')]


In [9]:
db_curr.execute("""CREATE TABLE delivery_persons (
    delivery_person_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50) NOT NULL,
    contact_number VARCHAR(15) NOT NULL UNIQUE,
    vehicle_type VARCHAR(20),
    total_deliveries INT NOT NULL DEFAULT 0,
    average_rating INT NOT NULL,
    location VARCHAR(100) NOT NULL
);""")
db_curr.execute("""DESC delivery_persons;""")
print(db_curr.fetchall())

[('delivery_person_id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('name', 'varchar(50)', 'NO', '', None, ''), ('contact_number', 'varchar(15)', 'NO', 'UNI', None, ''), ('vehicle_type', 'varchar(20)', 'YES', '', None, ''), ('total_deliveries', 'int', 'NO', '', '0', ''), ('average_rating', 'int', 'NO', '', None, ''), ('location', 'varchar(100)', 'NO', '', None, '')]


In [10]:
db_curr.execute("""CREATE TABLE deliveries (
    delivery_id INT PRIMARY KEY AUTO_INCREMENT,
    order_id INT NOT NULL,
    delivery_person_id INT,
    delivery_status VARCHAR(15) NOT NULL,
    distance INT NOT NULL CHECK (distance > 0),
    delivery_time INT NOT NULL,
    estimated_time INT NOT NULL,
    delivery_fee INT NOT NULL,
    vehicle_type VARCHAR(20) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id) ON DELETE CASCADE,
    FOREIGN KEY (delivery_person_id) REFERENCES delivery_persons(delivery_person_id) ON DELETE SET NULL
)""")
db_curr.execute("""DESC delivery_persons;""")
print(db_curr.fetchall())

[('delivery_person_id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('name', 'varchar(50)', 'NO', '', None, ''), ('contact_number', 'varchar(15)', 'NO', 'UNI', None, ''), ('vehicle_type', 'varchar(20)', 'YES', '', None, ''), ('total_deliveries', 'int', 'NO', '', '0', ''), ('average_rating', 'int', 'NO', '', None, ''), ('location', 'varchar(100)', 'NO', '', None, '')]


Data Population using Faker

In [13]:
# Insert data into tables
fake = Faker()

num_of_rows=100

def insert_into_DB(table_name, dict_values):
    columns = ", ".join(dict_values.keys())
    placeholders = ", ".join(["%s"] * len(dict_values))
    sql_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    values = tuple(dict_values.values())
    try:
        db_curr.execute(sql_query, values)
        cnx.commit()
        #print("Order inserted successfully!")
    except db.Error as err:
        print(f"Error: {err}")
        cnx.rollback()
    

In [12]:
# Insert into customers table
for _ in range(num_of_rows):
    customer = {
        "customer_id": fake.unique.random_int(min=111111, max=999999),
        "name": fake.name(),
        "email": f"{fake.first_name().lower()}.{fake.last_name().lower()}@{fake.free_email_domain()}",
        "phone": f"{random.choice(['+1', '+44', '+91', '+61', '+81', '+49', '+33', '+55', '+86', '+7'])} {fake.numerify('##########')}",
        "location": fake.address(),
        "signup_date": fake.date_between(start_date='-2y', end_date='today'),
        "is_premium": random.choice([True, False]),
        "preferred_cuisine": random.choice(["Mexican", "Indian", "Italian", "American", "Japanese", "Thai", "French"]),
        "total_orders": fake.random_int(min=0, max=12),
        "average_rating": fake.random_int(min=1, max=5)
    }
    insert_into_DB("customers", customer)

Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted successfully!
Order inserted

In [14]:
# Insert into restaurants table
for _ in range(num_of_rows):
    restaurant = {
        "restaurant_id": ''.join(fake.random_letters(length=3)).upper() + str(fake.unique.random_int(min=111111, max=999999)),
        "name": fake.company(),
        "cuisine_type": random.choice(["Mexican", "Indian", "Italian", "American", "Japanese", "Thai", "French"]),
        "location": fake.address(),
        "owner_name": fake.name(),
        "average_delivery_time": fake.random_int(min=10, max=90),
        "contact_number": f"{random.choice(['+1', '+44', '+91', '+61', '+81', '+49', '+33', '+55', '+86', '+7'])}{fake.numerify('##########')}",
        "rating": fake.random_int(min=1, max=5),
        "total_orders": fake.random_int(min=0, max=50),
        "is_active": random.choice([True, False])
    }
    insert_into_DB("restaurants", restaurant)

In [15]:
# Insert into orders table
db_curr.execute("SELECT customer_id FROM customers;")
customers = [row[0] for row in db_curr.fetchall()]
db_curr.execute("SELECT restaurant_id FROM restaurants;")
restaurants = [row[0] for row in db_curr.fetchall()]

for _ in range(num_of_rows):
    order_date = fake.date_time_between(start_date='-20y')
    total_amount = fake.random_number(digits=5, fix_len=False)
    discount_applied = total_amount * (fake.random_int(min=0, max=80, step=5) / 100)
    order = {
        "order_id": fake.unique.random_int(min=11111111, max=99999999),
        "customer_id": random.choice(customers),
        "restaurant_id": random.choice(restaurants),
        "order_date": order_date,
        "delivery_time": order_date + timedelta(days=fake.random_int(min=2, max=5)),
        "status": random.choice(["Pending", "Delivered", "Cancelled"]),
        "total_amount": total_amount,
        "payment_mode": random.choice(["Credit Card", "Cash", "UPI"]),
        "discount_applied": discount_applied,
        "feedback_rating": fake.random_int(min=1, max=5)
    }
    insert_into_DB("orders", order)

In [16]:
# Insert into delivery_persons table
for _ in range(num_of_rows):
    delivery_person = {
        "delivery_person_id": fake.unique.random_int(min=111111, max=999999),
        "name": fake.name(),
        "location": fake.address(),
        "contact_number": random.choice(['+1', '+44', '+91', '+61', '+81', '+49', '+33', '+55', '+86', '+7']) + fake.numerify('##########'),
        "vehicle_type": random.choice(["Bike", "Cycle", "Scooter", "Van"]),
        "total_deliveries": random.randint(0, 50),
        "average_rating": random.randint(1, 5)
    }
    insert_into_DB("delivery_persons", delivery_person)

In [17]:
# Insert into deliveries table
db_curr.execute("SELECT order_id FROM orders;")
order_ids = [row[0] for row in db_curr.fetchall()]
db_curr.execute("SELECT delivery_person_id FROM delivery_persons;")
delivery_person_ids = [row[0] for row in db_curr.fetchall()]

for _ in range(num_of_rows):
    delivery = {
        "delivery_id": fake.unique.random_int(min=11111111, max=99999999),
        "order_id": random.choice(order_ids),
        "delivery_person_id": random.choice(delivery_person_ids),
        "delivery_status": fake.random_element(elements=["On the way", "Delivered"]),
        "distance": random.randint(2, 10),
        "delivery_time": random.randint(10, 90),
        "estimated_time": random.randint(10, 100),
        "delivery_fee": fake.random_int(min=10, max=40, step=10),
        "vehicle_type": random.choice(["Bike", "Cycle", "Scooter", "Van"])
    }
    insert_into_DB("deliveries", delivery)

In [18]:
db_curr.close()
cnx.close()
print("Database connection closed.")

Database connection closed.
